## Check to collect sites

In [ ]:
from urllib.parse import urlparse

def clean_url(url):
    parsed_url = urlparse(url)
    netloc_without_www = parsed_url.netloc.lstrip('www.')
    cleaned_url = netloc_without_www + parsed_url.path
    
    if cleaned_url.endswith('/'):
        cleaned_url = cleaned_url[:-1]
    if cleaned_url.startswith("www."):
        cleaned_url = cleaned_url[4:]
    return cleaned_url

df_sw = pd.read_csv("data/similarWeb_ranking_2023-6.csv")

allsides_urls = [clean_url(url) for url in df["link"]]
similarwebs_urls = [clean_url(url) for url in df_sw["Website"]]

In [ ]:
df_additional = df_sw[df_sw["Website"].isin(list(set(similarwebs_urls) - set(allsides_urls)))]
df_additional = df_additional.drop(columns = ["Unnamed: 0"])

In [ ]:
df_additional["info"] = "N/A"
df_additional.to_csv("additonal_results.csv")

## Contstuct links to be collected

In [ ]:
# prevent function from access global namespace
import types
noglobal = lambda f: types.FunctionType(f.__code__, {}, argdefs=f.__defaults__)

In [ ]:
@noglobal
def construct_json_url(url, matchType, limit, _filter, _from, _to, _collapse):
    link = f"http://web.archive.org/cdx/search/cdx?url={url}&matchType={matchType}&limit={limit}"\
        + f"&filter={_filter}&from={_from}&to={_to}&collapse={_collapse}"

    return link

In [ ]:
url = "https://www.dailymail.co.uk"
matchType = "exact"
limit = "10"
_filter = "statuscode:301|302"
_from = 2022
_to= 2023
_collapse = "timestamp:10"
    
print(construct_json_url(url, matchType, limit, _filter, _from, _to, _collapse))

In [ ]:
url = "https://web.archive.org/cdx/search/cdx?url=https://www.dailymail.co.uk&matchType=exact&limit=13500&from=2022&to=2023&collapse=timestamp:10"
file_name = "Daily-Mail_13500_202201_202306.txt"

r = get_link_with_proxy(url)
with open("data/site_links/" + file_name, "w") as outfile:
    outfile.write(r.text)

## Collect links

In [ ]:
matchType = "exact"
limit = "13500" # 13500
_filter = "statuscode:200"
_from = 202201
_to= 202306
_collapse = "timestamp:10"
    
for x in df.iterrows():
    x = x[1]
    url = x["link"]
    name = x["news"]
    
    name = name.replace(" ", "-")
    file_name = f"{name}_{limit}_{_from}_{_to}.txt"
    print(file_name)
    
    file_url = construct_json_url(url, matchType, limit, _filter, _from, _to, _collapse)
    r = get_link_with_proxy(file_url)
    
    with open("data/site_links/" + file_name, "w") as outfile:
        outfile.write(r.text)

In [ ]:
with open("data/site_links/test.txt", "w") as outfile:
    outfile.write(r.text)

### select datepoint to collect